In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Lasso, Ridge, ElasticNet, LinearRegression
from sklearn.preprocessing import FunctionTransformer, KBinsDiscretizer, StandardScaler, MinMaxScaler, PolynomialFeatures, RobustScaler, OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
from sklearn.pipeline import make_pipeline
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer


In [2]:
data_path = "Dataset_modelisation.csv"
df = pd.read_csv(data_path)

In [3]:
X = df.drop('charges', axis=1)
y = df.charges

X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, train_size=0.85, random_state=42, stratify=X['smoker'])

num_col = list(X.select_dtypes(include=[float,int]).columns)
cat_col = list(X.select_dtypes(include=[object]).columns)

onehotscale_pipeline = make_pipeline(OneHotEncoder(), RobustScaler(with_centering=False))
scale_pipeline = make_pipeline(RobustScaler(with_centering=False))

preprocessing = ColumnTransformer(
    transformers=[
        ('categorical', onehotscale_pipeline, cat_col),
        ('numerical', scale_pipeline, num_col)]
)

polyscale_pipeline = make_pipeline(PolynomialFeatures(2))

my_final_pipeline = make_pipeline(preprocessing, polyscale_pipeline)
my_final_pipeline.fit(X_train)

feature_names = my_final_pipeline.get_feature_names_out(X.columns)

model = make_pipeline(
    my_final_pipeline,
    Lasso(random_state=42, max_iter=100000)
)

param_grid = {'lasso__alpha': [15]}


model = GridSearchCV(model, param_grid,cv =5)
model.fit(X_train, y_train)
grid_score = model.score(X_train, y_train)

best_alpha = model.best_params_['lasso__alpha']

y_pred_train = model.predict(X_train)


In [4]:

residus = y_train - y_pred_train

weights = np.exp(-abs(residus) / residus.std())
model.fit(X_train, y_train, lasso__sample_weight=weights)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('pipeline',
                                        Pipeline(steps=[('columntransformer',
                                                         ColumnTransformer(transformers=[('categorical',
                                                                                          Pipeline(steps=[('onehotencoder',
                                                                                                           OneHotEncoder()),
                                                                                                          ('robustscaler',
                                                                                                           RobustScaler(with_centering=False))]),
                                                                                          ['sex',
                                                                                           'smoker',
                                                                                           'region',
                                                                                           'imc_category']),
                                                                                         ('numerical',
                                                                                          Pipeline(steps=[('robustscaler',
                                                                                                           RobustScaler(with_centering=False))]),
                                                                                          ['age',
                                                                                           'children'])])),
                                                        ('pipeline',
                                                         Pipeline(steps=[('polynomialfeatures',
                                                                          PolynomialFeatures())]))])),
                                       ('lasso',
                                        Lasso(max_iter=100000,
                                              random_state=42))]),
             param_grid={'lasso__alpha': [15]})

In [5]:
y_pred = model.predict(X_test)

r2_cleaned = r2_score(y_test, y_pred)
mse_cleaned = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse_cleaned = np.sqrt(mse_cleaned)

print("Performance du modèle après suppression des valeurs extrêmes:")
print(model.best_params_)
print(f"R2 Score: {r2_cleaned:.4f}")
print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse_cleaned:.2f}")

Performance du modèle après suppression des valeurs extrêmes:
{'lasso__alpha': 15}
R2 Score: 0.9279
MAE: 982.60
RMSE: 3232.04


In [6]:
with open('modele.pkl', 'wb') as file:
    pickle.dump(model, file)